# Flask Playground

### This notebook containers Selenium Simulation Code that would preform the data capture from ecWIN.

In [276]:
import sys
import json
import datetime
import time

import pprint
pp = pprint.PrettyPrinter(indent=4)

from pymongo import MongoClient
from bson.objectid import ObjectId
from influxdb import InfluxDBClient
from influxdb import DataFrameClient


import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt

import time
import datetime

from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from difflib import get_close_matches
import random

## Configure Mongodb

In [2]:
mongoClient = MongoClient('localhost',27017)
db = mongoClient.witsECOMP

## Configure InfluxDB

In [218]:
databaseName = "timeseriesdata"
influxClient = DataFrameClient("localhost", 8086, "root", "root", database=databaseName)
influxClient.create_database(databaseName)

In [217]:
influxClient.drop_database(databaseName)

## Read in time Series from excel spreadsheet
Note: this process can take a long time (2 to 3 minutes!).

In [4]:
spreadsheet = pd.read_excel('/home/chris/Desktop/Wits Data/WITS ecWIN7 Data/WITS ecWIN7 Data 2018.xlsx',
                            sheet_name='WITS ecWIN7 Data 2018',
                            index_col="DateTime"
                           )

## Read in building names and geojson information
These files were manually created by generating geojson from http://geojson.io/ and identifying the corisponding building names.

In [83]:
buildingSensorNames = spreadsheet.columns
print(buildingSensorNames)

Index(['WITS 13 Jubilee Road_kVA', 'WITS 13 Jubilee Road_kVarh',
       'WITS 13 Jubilee Road_kWh', 'WITS 3 Jubilee Road_kVA',
       'WITS 3 Jubilee Road_kVarh', 'WITS 3 Jubilee Road_kWh',
       'WITS 5 Jubilee Road_kVA', 'WITS 5 Jubilee Road_kVarh',
       'WITS 5 Jubilee Road_kWh', 'WITS 9 Jubilee Road_kVA',
       ...
       'WITS WC Village Zesti Lemonz_kWh', 'WITS WC WITS Club GEN 100 kVa_kVA',
       'WITS WC WITS CLUB GENERATOR 100 kVa_kVarh',
       'WITS WC WITS CLUB GENERATOR 100 kVa_kWh', 'WITS WC Wits Club_kVA',
       'WITS WC Wits Club_kVarh', 'WITS WC Wits Club_kWh',
       'WITS Wits Club Summation_kVA', 'WITS Wits Club Summation_kVarh',
       'WITS Wits Club Summation_kWh'],
      dtype='object', length=986)


In [84]:
loadedGeoJson = {}
with open('../../Assets/geojson/witsMainCampusGeojson.json') as json_data:
    loadedGeoJson = json.load(json_data)
    print(loadedGeoJson)

{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'stroke': '#555555', 'stroke-width': 2, 'stroke-opacity': 1, 'fill': '#555555', 'fill-opacity': 0.5, 'buildingName': 'Chamber of Mines'}, 'geometry': {'type': 'Polygon', 'coordinates': [[[28.026670217514038, -26.19134352734578], [28.026729226112362, -26.191767125729683], [28.026643395423886, -26.191786380165073], [28.026697039604187, -26.19204150113358], [28.0270779132843, -26.19199336514455], [28.027067184448242, -26.19195485633902], [28.02739977836609, -26.19192116112372], [28.027292490005493, -26.191271322922137], [28.026670217514038, -26.19134352734578]]]}}, {'type': 'Feature', 'properties': {'stroke': '#555555', 'stroke-width': 2, 'stroke-opacity': 1, 'fill': '#555555', 'fill-opacity': 0.5, 'buildingName': 'Flower Hall'}, 'geometry': {'type': 'Polygon', 'coordinates': [[[28.025929927825928, -26.19151200416024], [28.025962114334106, -26.191950042737417], [28.026278614997864, -26.191901906710605], [28.0261

In [219]:
for index, feature in enumerate(loadedGeoJson["features"]):
    buildingName = feature["properties"]["buildingName"]
    mostLiklySensor = get_close_matches(buildingName + "_kVA", buildingSensorNames)[0]
    dataFrame = spreadsheet[mostLiklySensor].to_frame()
    influxClient.write_points(dataFrame, databaseName,{'buildingNumber': index, 'buildingName': buildingName, 'sensorName': mostLiklySensor})
    print(buildingName + "->" + mostLiklySensor)

Chamber of Mines->WITS WC Chamber of Mines Total_kVA
Flower Hall->WITS WC Flower Hall_kVA
Ithemba Labs High Voltage->WITS Ithemba Labs High Voltage_kVA
Wits Science Stadium->WITS WC Science Staduim TRF 2_kVA
Mathematical Science Building->WITS WC Maths Science Building_kVA
Zesti Lemonz->WITS EC PDH Zesti Lemon_kVA
FNB Building->WITS WC FNB Building Total_kVA
PEC PIMD Office Block->WITS PEC PIMD Office Block_kVA
DJ Du Plesis->WITS WC DJ du Plessis Building_kVA
West Campus Village->WITS Campus Lodge_kVA
Convocation Dining Hall->WITS WC Convocation Dining Hall GEN 125 kVA_kVA
Barnato Residence->WITS WC Barnato Sub Residence A - D_kVA
David Webster Hall->WITS WC David Webster Hall_kVA
The Barns->WITS WC The Barns_kVA
Olives and Plates->WITS WBS Olives & Plates Meter 2_kVA
Johannesburg Planetarium->WITS EC Planetarium_kVA
Wartenweilier Library->WITS EC Wartenweiler Library_kVA
Origins Centre->WITS EC Origin Centre_kVA
Chemistry Building->WITS EC Physics Building_kVA
Physics Building->WITS E

## Preform Queries to load agrigated data into the mongoDB.

These queries MUST pull from the influxdb as this is where all the time series data is stored. As new records are added from Selenium (or directly from tnew sensors) they will be added to the Influx DB. this querying and updating mongo process occures every 30 minutes to generate "fresh" graphs.

First, we need to make the database store for each building

In [340]:
buildingInformationStore = db.buildingInformation

Then, itterate over all the buildings that have been identified in the geojson and query for values for that building from influxdb for the predefined length of time. For each element, influx db is queried and then the output is computed over to generate averages per day, week and year. The maximum building week is also extracted. This value is used later to generate the heat map.


In [346]:
startTime = "2018-08-19T00:00:00+00:00"
plotsToDraw = {
    'Day':{
        "resampleLength":'1H',
        "windowLength": 2*24 #1 day window with 2 samples per hour
        },
    'Week':{
        "resampleLength":'1D',
        "windowLength":2*24*7 # 1 week window with 2 samples per hour
        },
    'Year':{
        "resampleLength":'1M',
        "windowLength":2*24*240 #1 year window with 2 samples per hour
        },
}

maxBuildingWeek = 0

for index, feature in enumerate(loadedGeoJson["features"]):
    buildingName = feature["properties"]["buildingName"]
    
    print(buildingName)
    buildingBlob = {
        "buildingID": index,
        "buildingName": buildingName,
        "sensorName": "SENSOR",
        "chartInformation": {
            "DayInformation": {
                "LastDay": [],
                "LastDayAverage":0,
                "AverageDay":[]
            },
            "WeekInformation": {
                "LastWeek": [],
                "LastWeekAverage": 0,
                "AverageWeek":[]
            },
            "YearInformation": {          
                "LastYear": [],
                "LastYearAverage":0,
                "AverageYear":[]
            }
        },
    }
    for plotType in plotsToDraw:
        print(plotsToDraw[plotType]['resampleLength'], plotsToDraw[plotType]['windowLength'])
        query = "SELECT * FROM timeseriesdata WHERE buildingName='{}' AND time< '{}'".format(buildingName, startTime)
        queryResults = influxClient.query(query)
        results = queryResults["timeseriesdata"][queryResults["timeseriesdata"].keys()[0]]
#         print(results.tail(plotsToDraw[plotType]['windowLength']))
        resampledResults = results.tail(plotsToDraw[plotType]['windowLength']).resample(plotsToDraw[plotType]['resampleLength'], label='right').sum()
#         print(resampledResults.to_dict())
        print("\n")
        buildingBlob["chartInformation"][plotType+"Information"]["Last" + plotType + "Average"]=resampledResults.mean()
        resultsDict = resampledResults.to_dict()
        formattedDict = {}
        formattedDict_Average = {}
        for result in resultsDict:
            formattedDict[str(result)[0:19]] = resultsDict[result]
            formattedDict_Average[str(result)[0:19]] = resultsDict[result]*random.randint(95,105)/100
        buildingBlob["chartInformation"][plotType+"Information"]["Last" + plotType] = formattedDict
        buildingBlob["chartInformation"][plotType+"Information"]["Average" + plotType]=formattedDict_Average
        
        
    if(buildingBlob["chartInformation"]["WeekInformation"]["LastWeekAverage"]>maxBuildingWeek):
        maxBuildingWeek=buildingBlob["chartInformation"]["WeekInformation"]["LastWeekAverage"]
        
    buildingInformation_id = buildingInformationStore.insert_one(buildingBlob)
#     print (buildingInformation_id)
    
print(maxBuildingWeek)
# dataFrame = pd.DataFrame.from_dict(resultsDict)
# print(dataFrame)

Chamber of Mines
1H 48


1D 336


1M 11520


{'buildingID': 0, 'buildingName': 'Chamber of Mines', 'sensorName': 'SENSOR', 'chartInformation': {'DayInformation': {'LastDay': {'2018-08-18 01:00:00': 145.17023441840848, '2018-08-18 02:00:00': 142.2692258762089, '2018-08-18 03:00:00': 137.0727357562059, '2018-08-18 04:00:00': 136.55271513839682, '2018-08-18 05:00:00': 133.15015538115819, '2018-08-18 06:00:00': 134.2050197953401, '2018-08-18 07:00:00': 138.76943992972662, '2018-08-18 08:00:00': 183.57798103309602, '2018-08-18 09:00:00': 197.9376369596955, '2018-08-18 10:00:00': 216.354603149402, '2018-08-18 11:00:00': 213.302910783575, '2018-08-18 12:00:00': 220.675301402591, '2018-08-18 13:00:00': 215.06393790736, '2018-08-18 14:00:00': 178.6469660672619, '2018-08-18 15:00:00': 176.0618312497865, '2018-08-18 16:00:00': 168.9473004688703, '2018-08-18 17:00:00': 173.4287448240393, '2018-08-18 18:00:00': 171.9102988910057, '2018-08-18 19:00:00': 173.7684152994556, '2018-08-18 20:00:00': 172.



1D 336


1M 11520


{'buildingID': 3, 'buildingName': 'Wits Science Stadium', 'sensorName': 'SENSOR', 'chartInformation': {'DayInformation': {'LastDay': {'2018-08-18 01:00:00': 39.2399761024674, '2018-08-18 02:00:00': 39.051382729773195, '2018-08-18 03:00:00': 39.803846483734006, '2018-08-18 04:00:00': 39.0780771082133, '2018-08-18 05:00:00': 40.9654331564233, '2018-08-18 06:00:00': 38.3343270082759, '2018-08-18 07:00:00': 37.4422465469373, '2018-08-18 08:00:00': 37.290979144444194, '2018-08-18 09:00:00': 37.6005487664839, '2018-08-18 10:00:00': 37.2352558929625, '2018-08-18 11:00:00': 38.7659803285528, '2018-08-18 12:00:00': 42.1963961038724, '2018-08-18 13:00:00': 47.2156038726074, '2018-08-18 14:00:00': 50.2827529865914, '2018-08-18 15:00:00': 51.519631667508804, '2018-08-18 16:00:00': 52.3267450104468, '2018-08-18 17:00:00': 48.6037677777586, '2018-08-18 18:00:00': 52.867249250773696, '2018-08-18 19:00:00': 47.9117650021226, '2018-08-18 20:00:00': 45.826136157463395, '2018-08-18 



1D 336


1M 11520


{'buildingID': 6, 'buildingName': 'FNB Building', 'sensorName': 'SENSOR', 'chartInformation': {'DayInformation': {'LastDay': {'2018-08-18 01:00:00': 102.4733337330408, '2018-08-18 02:00:00': 100.5279191676115, '2018-08-18 03:00:00': 104.9904812467781, '2018-08-18 04:00:00': 76.91801802240221, '2018-08-18 05:00:00': 69.80155409286749, '2018-08-18 06:00:00': 65.5066613501408, '2018-08-18 07:00:00': 65.58246454993261, '2018-08-18 08:00:00': 66.2440474177038, '2018-08-18 09:00:00': 62.8729542093016, '2018-08-18 10:00:00': 68.5584975598172, '2018-08-18 11:00:00': 73.93602950376331, '2018-08-18 12:00:00': 76.1660559228503, '2018-08-18 13:00:00': 76.48785466028309, '2018-08-18 14:00:00': 74.2546363658779, '2018-08-18 15:00:00': 72.3458957404496, '2018-08-18 16:00:00': 80.03360126448709, '2018-08-18 17:00:00': 83.2793713248156, '2018-08-18 18:00:00': 83.4768242824626, '2018-08-18 19:00:00': 82.3667122554046, '2018-08-18 20:00:00': 85.6836845713464, '2018-08-18 21:00:00': 



1D 336


1M 11520


{'buildingID': 9, 'buildingName': 'West Campus Village', 'sensorName': 'SENSOR', 'chartInformation': {'DayInformation': {'LastDay': {'2018-08-18 01:00:00': 25.109926152535103, '2018-08-18 02:00:00': 24.4052657161055, '2018-08-18 03:00:00': 24.310274052643898, '2018-08-18 04:00:00': 26.1068957419683, '2018-08-18 05:00:00': 23.9053537327406, '2018-08-18 06:00:00': 26.409663852228803, '2018-08-18 07:00:00': 23.5106754147999, '2018-08-18 08:00:00': 31.5039997382276, '2018-08-18 09:00:00': 35.7089625307028, '2018-08-18 10:00:00': 38.926061411137795, '2018-08-18 11:00:00': 36.427176401484004, '2018-08-18 12:00:00': 45.1217371733647, '2018-08-18 13:00:00': 39.611902544805105, '2018-08-18 14:00:00': 34.7178628266493, '2018-08-18 15:00:00': 29.0068954368727, '2018-08-18 16:00:00': 34.637074534300396, '2018-08-18 17:00:00': 35.019617139295306, '2018-08-18 18:00:00': 36.5232357303034, '2018-08-18 19:00:00': 35.0036386643697, '2018-08-18 20:00:00': 33.0075278535155, '2018-08-



1D 336


1M 11520


{'buildingID': 12, 'buildingName': 'David Webster Hall', 'sensorName': 'SENSOR', 'chartInformation': {'DayInformation': {'LastDay': {'2018-08-18 01:00:00': 123.578456512275, '2018-08-18 02:00:00': 123.60752319865671, '2018-08-18 03:00:00': 112.77470934882379, '2018-08-18 04:00:00': 107.472053252538, '2018-08-18 05:00:00': 103.3020575349253, '2018-08-18 06:00:00': 107.22232813497439, '2018-08-18 07:00:00': 111.1484095244804, '2018-08-18 08:00:00': 108.94963185815149, '2018-08-18 09:00:00': 108.3189365453344, '2018-08-18 10:00:00': 111.8821832975438, '2018-08-18 11:00:00': 115.589143664808, '2018-08-18 12:00:00': 124.05108840610819, '2018-08-18 13:00:00': 131.9222283117503, '2018-08-18 14:00:00': 143.1042247650122, '2018-08-18 15:00:00': 145.65575825715882, '2018-08-18 16:00:00': 159.8675574980306, '2018-08-18 17:00:00': 160.5443165160124, '2018-08-18 18:00:00': 164.56116511105802, '2018-08-18 19:00:00': 192.6822005097651, '2018-08-18 20:00:00': 199.5728981257082, '



1D 336


1M 11520


{'buildingID': 15, 'buildingName': 'Johannesburg Planetarium', 'sensorName': 'SENSOR', 'chartInformation': {'DayInformation': {'LastDay': {'2018-08-18 01:00:00': 15.36, '2018-08-18 02:00:00': 17.119999999999997, '2018-08-18 03:00:00': 15.52, '2018-08-18 04:00:00': 13.600000000000001, '2018-08-18 05:00:00': 16.0, '2018-08-18 06:00:00': 15.84, '2018-08-18 07:00:00': 18.08, '2018-08-18 08:00:00': 35.68, '2018-08-18 09:00:00': 15.36, '2018-08-18 10:00:00': 19.04, '2018-08-18 11:00:00': 24.479999999999997, '2018-08-18 12:00:00': 25.12, '2018-08-18 13:00:00': 21.6, '2018-08-18 14:00:00': 19.68, '2018-08-18 15:00:00': 16.8, '2018-08-18 16:00:00': 18.240000000000002, '2018-08-18 17:00:00': 20.0, '2018-08-18 18:00:00': 40.64, '2018-08-18 19:00:00': 56.0, '2018-08-18 20:00:00': 58.400000000000006, '2018-08-18 21:00:00': 55.84, '2018-08-18 22:00:00': 55.68, '2018-08-18 23:00:00': 57.120000000000005, '2018-08-19 00:00:00': 55.84}, 'LastDayAverage': 29.459999999999997, 'Averag



1D 336


1M 11520


{'buildingID': 18, 'buildingName': 'Chemistry Building', 'sensorName': 'SENSOR', 'chartInformation': {'DayInformation': {'LastDay': {'2018-08-18 01:00:00': 57.4943439460287, '2018-08-18 02:00:00': 55.1638081509812, '2018-08-18 03:00:00': 55.9195724137737, '2018-08-18 04:00:00': 55.0484971096017, '2018-08-18 05:00:00': 56.4587052621179, '2018-08-18 06:00:00': 55.9886704019381, '2018-08-18 07:00:00': 58.5624213325587, '2018-08-18 08:00:00': 59.1137098496619, '2018-08-18 09:00:00': 58.2413064125418, '2018-08-18 10:00:00': 58.577373942067396, '2018-08-18 11:00:00': 58.6104016728293, '2018-08-18 12:00:00': 60.864039595097495, '2018-08-18 13:00:00': 61.786508445273, '2018-08-18 14:00:00': 61.201139391172006, '2018-08-18 15:00:00': 60.0647914755658, '2018-08-18 16:00:00': 61.9918549712905, '2018-08-18 17:00:00': 64.9071722756276, '2018-08-18 18:00:00': 65.6126975849131, '2018-08-18 19:00:00': 61.431792327152195, '2018-08-18 20:00:00': 61.671348337558, '2018-08-18 21:00:0



1D 336


1M 11520


{'buildingID': 21, 'buildingName': 'William Cullen Library', 'sensorName': 'SENSOR', 'chartInformation': {'DayInformation': {'LastDay': {'2018-08-18 01:00:00': 5.20246212391752, '2018-08-18 02:00:00': 5.471190105512591, '2018-08-18 03:00:00': 5.15481347759245, '2018-08-18 04:00:00': 5.27081119599825, '2018-08-18 05:00:00': 5.05245998104497, '2018-08-18 06:00:00': 5.00644792193681, '2018-08-18 07:00:00': 5.47333316932379, '2018-08-18 08:00:00': 4.7005101509685705, '2018-08-18 09:00:00': 5.263026330076849, '2018-08-18 10:00:00': 4.55269480573436, '2018-08-18 11:00:00': 4.60108734169401, '2018-08-18 12:00:00': 5.016964862553859, '2018-08-18 13:00:00': 4.55109890105085, '2018-08-18 14:00:00': 4.65267011133329, '2018-08-18 15:00:00': 5.21574597618142, '2018-08-18 16:00:00': 4.85208242989286, '2018-08-18 17:00:00': 4.65887440274384, '2018-08-18 18:00:00': 4.65695525450759, '2018-08-18 19:00:00': 4.35558961051594, '2018-08-18 20:00:00': 4.45278923437758, '2018-08-18 21:0



1D 336


1M 11520


{'buildingID': 24, 'buildingName': 'Gym and Squash courts', 'sensorName': 'SENSOR', 'chartInformation': {'DayInformation': {'LastDay': {'2018-08-18 01:00:00': 43.9741014689328, '2018-08-18 02:00:00': 35.4293519867057, '2018-08-18 03:00:00': 36.8643652466627, '2018-08-18 04:00:00': 27.407024083552102, '2018-08-18 05:00:00': 31.5242727946364, '2018-08-18 06:00:00': 28.2956601742645, '2018-08-18 07:00:00': 40.8427849453023, '2018-08-18 08:00:00': 38.5015101546673, '2018-08-18 09:00:00': 50.4339876282544, '2018-08-18 10:00:00': 43.464560084987795, '2018-08-18 11:00:00': 54.8413861853656, '2018-08-18 12:00:00': 56.6575240366456, '2018-08-18 13:00:00': 62.7004297550608, '2018-08-18 14:00:00': 62.6908326219945, '2018-08-18 15:00:00': 58.7133059037746, '2018-08-18 16:00:00': 49.3104264087424, '2018-08-18 17:00:00': 34.5834410794793, '2018-08-18 18:00:00': 42.3151006494022, '2018-08-18 19:00:00': 26.6066959441442, '2018-08-18 20:00:00': 35.6727934617927, '2018-08-18 21:00:

NameError: name 'maxBuildingDay' is not defined

## Generate heatmap
The next step is to generate the heat map based off the magnitudes of the buildings. A function is defined that converts a minimum, maximum and value to an rgb hex value.

In [348]:
def rgb(minimum, maximum, value):
    minimum, maximum = float(minimum), float(maximum)
    ratio = 2 * (value-minimum) / (maximum - minimum)
    b = int(max(0, 255*(1 - ratio)))
    r = int(max(0, 255*(ratio - 1)))
    g = 255 - b - r
    return r, g, b

def getHex(minimum,maximum,value):
    return '#%02x%02x%02x' % rgb(minimum,maximum,value)

Next, we itterate over the geojson set and update the square heat based of the set maximum

In [365]:
for index, feature in enumerate(loadedGeoJson["features"]):
    buildingName = feature["properties"]["buildingName"]
    buildingValueWeek = buildingInformationStore.find_one({"buildingName": buildingName})['chartInformation']['WeekInformation']['LastWeekAverage']
    newColour = getHex(0,maxBuildingWeek, buildingValueWeek)
    loadedGeoJson["features"][index]["properties"]["fill"] = newColour
    loadedGeoJson["features"][index]["properties"]["fill"]
    

# Incert the geojson into the database
Now that the heatmap colours have been calculated, we can incert it into the database

In [366]:
geojsonStore = db.geojson
geojson_id = geojsonStore.insert_one(loadedGeoJson).inserted_id

In [281]:
loadedCampusInfo = {}
with open('../../Assets/otherinformation/campusInfomation.json') as json_data:
    loadedCampusInfo = json.load(json_data)
    print(loadedCampusInfo)

{'name': 'Main campus', 'center': ['28.0240, -26.18701'], 'numberOfStudents': 25000, 'averageWeekChart': {}, 'piDistribution': {}, 'sunburstDistribution': {}, 'streamGraph': {}, 'totalSensors': 2000}


In [11]:
campusInfoStore = db.campusInfo
campusInfo_id = campusInfoStore.insert_one(loadedCampusInfo).inserted_id

NameError: name 'db' is not defined

In [7]:
list(campusInfoStore.find({}))

[{'_id': ObjectId('5bbb60adb2831017c6117dbe'),
  'name': 'Main campus',
  'center': ['28.0240, -26.18701'],
  'numberOfStudents': 25000,
  'averageWeekChart': {},
  'piDistribution': {},
  'sunburstDistribution': {},
  'streamGraph': {},
  'totalSensors': 2000}]

In [8]:
loadedGeoJson = {}
with open('../../Assets/geojson/witsMainCampusGeojson.json') as json_data:
    loadedGeoJson = json.load(json_data)
    print(loadedGeoJson)

{'features': [{'geometry': {'coordinates': [[[28.026670217514038, -26.19134352734578], [28.026729226112362, -26.191767125729683], [28.026643395423886, -26.191786380165073], [28.026697039604187, -26.19204150113358], [28.0270779132843, -26.19199336514455], [28.027067184448242, -26.19195485633902], [28.02739977836609, -26.19192116112372], [28.027292490005493, -26.191271322922137], [28.026670217514038, -26.19134352734578]]], 'type': 'Polygon'}, 'properties': {'id': '0', 'buildingName': 'NOT Chamber Of Mines', 'style': {'color': '#999', 'fillColor': '#E04E5C', 'fillOpacity': 0.7, 'opacity': 1, 'weight': 2}}, 'type': 'Feature'}, {'geometry': {'coordinates': [[[28.025929927825928, -26.19151200416024], [28.025962114334106, -26.191950042737417], [28.026278614997864, -26.191901906710605], [28.026192784309387, -26.19148312243793], [28.025929927825928, -26.19151200416024]]], 'type': 'Polygon'}, 'properties': {'id': '1', 'buildingName': 'Chamber Of Mines', 'style': {'color': '#999', 'fillColor': '#

In [9]:
geojsonStore = db.geojson
geojson_id = geojsonStore.insert_one(loadedGeoJson).inserted_id

In [10]:
list(geojsonStore.find({}))

[{'_id': ObjectId('5bbb60adb2831017c6117dbf'),
  'features': [{'geometry': {'coordinates': [[[28.026670217514038,
        -26.19134352734578],
       [28.026729226112362, -26.191767125729683],
       [28.026643395423886, -26.191786380165073],
       [28.026697039604187, -26.19204150113358],
       [28.0270779132843, -26.19199336514455],
       [28.027067184448242, -26.19195485633902],
       [28.02739977836609, -26.19192116112372],
       [28.027292490005493, -26.191271322922137],
       [28.026670217514038, -26.19134352734578]]],
     'type': 'Polygon'},
    'properties': {'id': '0',
     'buildingName': 'NOT Chamber Of Mines',
     'style': {'color': '#999',
      'fillColor': '#E04E5C',
      'fillOpacity': 0.7,
      'opacity': 1,
      'weight': 2}},
    'type': 'Feature'},
   {'geometry': {'coordinates': [[[28.025929927825928, -26.19151200416024],
       [28.025962114334106, -26.191950042737417],
       [28.026278614997864, -26.191901906710605],
       [28.026192784309387, -26.19

In [11]:
BuildingChartInformation = {}
with open('../../Assets/ChartInformation/BuildingChartInformation.json') as json_data:
    BuildingChartInformation = json.load(json_data)
    print(BuildingChartInformation)

{'buildingId': '0', 'dayInformation': {'LastDay': [20, 14, 25, 16, 18, 22, 19, 20, 14, 25, 16, 18, 22, 19, 20, 14, 25, 16, 18, 22, 19, 21, 23, 24], 'AverageDay': [20, 14, 25, 16, 18, 22, 19, 20, 14, 25, 16, 18, 22, 19, 20, 14, 25, 16, 18, 22, 19, 21, 23, 24], 'CampusAverageDay': [20, 14, 25, 16, 18, 22, 19, 20, 14, 25, 16, 18, 22, 19, 20, 14, 25, 16, 18, 22, 19, 21, 23, 24], 'CampusAverageDayNormalized': [20, 14, 25, 16, 18, 22, 19, 20, 14, 25, 16, 18, 22, 19, 20, 14, 25, 16, 18, 22, 19, 21, 23, 24]}, 'weekInformation': {'LastWeek': [20, 14, 25, 16, 18, 22, 19], 'AverageWeek': [19, 14, 22, 14, 16, 19, 15], 'CampusAverageWeek': [28, 22, 30, 28, 28, 31, 22], 'CampusAverageWeekNormalized': [25, 19, 25, 26, 26, 28, 18]}, 'yearInformation': {'LastYear': [20, 14, 25, 16, 18, 22, 19], 'AverageYear': [19, 14, 22, 14, 16, 19, 15], 'CampusAverageYear': [28, 22, 30, 28, 28, 31, 22], 'CampusAverageYearNormalized': [25, 19, 25, 26, 26, 28, 18]}, 'histogramInformation': {}}


In [12]:
buildingInformationStore = db.buildingInformation
buildingInformation_id = buildingInformationStore.insert_one(BuildingChartInformation).inserted_id

In [13]:
list(buildingInformationStore.find({"buildingId": "0"}))

[{'_id': ObjectId('5bbb60adb2831017c6117dc0'),
  'buildingId': '0',
  'dayInformation': {'LastDay': [20,
    14,
    25,
    16,
    18,
    22,
    19,
    20,
    14,
    25,
    16,
    18,
    22,
    19,
    20,
    14,
    25,
    16,
    18,
    22,
    19,
    21,
    23,
    24],
   'AverageDay': [20,
    14,
    25,
    16,
    18,
    22,
    19,
    20,
    14,
    25,
    16,
    18,
    22,
    19,
    20,
    14,
    25,
    16,
    18,
    22,
    19,
    21,
    23,
    24],
   'CampusAverageDay': [20,
    14,
    25,
    16,
    18,
    22,
    19,
    20,
    14,
    25,
    16,
    18,
    22,
    19,
    20,
    14,
    25,
    16,
    18,
    22,
    19,
    21,
    23,
    24],
   'CampusAverageDayNormalized': [20,
    14,
    25,
    16,
    18,
    22,
    19,
    20,
    14,
    25,
    16,
    18,
    22,
    19,
    20,
    14,
    25,
    16,
    18,
    22,
    19,
    21,
    23,
    24]},
  'weekInformation': {'LastWeek': [20, 14, 25, 16, 18, 22, 19],
  